# API DATABASES

Con la finalidad de reducir la capacidad de procesamiento se crean bases de datos individuales para las consultas que plantea el cliente. A continuacion se trabajan y dividen las nuevas bases de datos.

## Importar Archivos

In [1]:
import pandas as pd
import numpy as np
import json 
import ast 

Carga de bases de datos en CSV

In [2]:
data_steam_output = pd.read_csv('Datasets/data_steam_output.csv')
data_users_items = pd.read_csv('Datasets/data_users_items.csv')
data_users_revs = pd.read_csv('Datasets/data_users_revs.csv')
data_genre = pd.read_csv('Datasets/data_genre.csv')
data_specs = pd.read_csv('Datasets/data_specs.csv')

# 1er endpoint userdata

## Api_userdata - Process

DATA_STEAM_OUTPUT

In [3]:
data_steam_output.head()

,item_id,app_name,developer,publisher,release_date,price,early_access,url,reviews_url
0,761140.0,Lost Summoner Kitty,Kotoshiro,Kotoshiro,2018-01-04,4.99,0.0,http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...
1,643980.0,Ironbound,Secret Level SRL,"Making Fun, Inc.",2018-01-04,0.00,0.0,http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...
2,670290.0,Real Pool 3D - Poolians,Poolians.com,Poolians.com,2017-07-24,0.00,0.0,http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...
3,767400.0,弹炸人2222,彼岸领域,彼岸领域,2017-12-07,0.99,0.0,http://store.steampowered.com/app/767400/2222/,http://steamcommunity.com/app/767400/reviews/?...
4,773570.0,Log Challenge,NaN,NaN,NaN,2.99,0.0,http://store.steampowered.com/app/773570/Log_C...,http://steamcommunity.com/app/773570/reviews/?...


DATA_ITEMS_PRICE: Se prepera la base de datos de los precios.

In [4]:
data_items_price = data_steam_output.copy() #Se crea una copia del dataframe para trabajar con el.
data_items_price.drop(columns=['app_name', 'developer', 'publisher', 'release_date', 'early_access', 'url', 'reviews_url'],inplace=True)

In [5]:
data_items_price.head() #Dataframe de items y precios.

,item_id,price
0,761140.0,4.99
1,643980.0,0.00
2,670290.0,0.00
3,767400.0,0.99
4,773570.0,2.99


DATA_USERS_ITEMS

In [6]:
data_users_items.head()

,user_id,items_count,steam_id,user_index_count,item_id,item_name,playtime_forever,playtime_2weeks,user_url
0,76561197970982479,277,76561197970982479,1,10,Counter-Strike,6,0,http://steamcommunity.com/profiles/76561197970...
1,76561197970982479,277,76561197970982479,1,20,Team Fortress Classic,0,0,http://steamcommunity.com/profiles/76561197970...
2,76561197970982479,277,76561197970982479,1,30,Day of Defeat,7,0,http://steamcommunity.com/profiles/76561197970...
3,76561197970982479,277,76561197970982479,1,40,Deathmatch Classic,0,0,http://steamcommunity.com/profiles/76561197970...
4,76561197970982479,277,76561197970982479,1,50,Half-Life: Opposing Force,0,0,http://steamcommunity.com/profiles/76561197970...


DATA_USERS_REVS

In [7]:
data_users_revs.head()

,user_id,item_id,recommend,sentiment_analysis,rev_date
0,76561197970982479,1250,True,1,2011-11-05
1,76561197970982479,22200,True,2,2011-07-15
2,76561197970982479,43110,True,1,2011-04-21
3,js41637,251610,True,1,2014-06-24
4,js41637,227300,True,1,2013-09-08


DATA_USERS: Se crea un dataframe aparte para luego entrecruzar datos

In [8]:
#Se copia el dataframe 'data_users_items' y se eliminan las columnas que no son necesarias para esta consulta de la API
data_users = data_users_items.copy()
data_users.drop(columns=['item_name', 'user_index_count' ,'steam_id', 'playtime_forever', 'playtime_2weeks'], inplace=True)

#Se unen las columnas del dataframe a traves de la columna 'item_id'.
data_users = data_users.merge(data_items_price, on='item_id', how='left')

In [9]:
data_users.head()

,user_id,items_count,item_id,user_url,price
0,76561197970982479,277,10,http://steamcommunity.com/profiles/76561197970...,9.99
1,76561197970982479,277,20,http://steamcommunity.com/profiles/76561197970...,4.99
2,76561197970982479,277,30,http://steamcommunity.com/profiles/76561197970...,4.99
3,76561197970982479,277,40,http://steamcommunity.com/profiles/76561197970...,4.99
4,76561197970982479,277,50,http://steamcommunity.com/profiles/76561197970...,4.99


In [10]:
#Ver los value counts de la columna price que NO sean Numeros
data_users['price'].value_counts()

price
9.99      865756
0.00      811332
19.99     803424
14.99     477380
4.99      331480
           ...  
124.99         2
1.50           1
119.99         1
74.99          1
2.89           1
Name: count, Length: 87, dtype: int64

In [11]:
#ver valores Nan
data_users[data_users['price'].isnull()]

,user_id,items_count,item_id,user_url,price
13,76561197970982479,277,34440,http://steamcommunity.com/profiles/76561197970...,NaN
21,76561197970982479,277,340,http://steamcommunity.com/profiles/76561197970...,NaN
26,76561197970982479,277,9340,http://steamcommunity.com/profiles/76561197970...,NaN
27,76561197970982479,277,228200,http://steamcommunity.com/profiles/76561197970...,NaN
37,76561197970982479,277,17340,http://steamcommunity.com/profiles/76561197970...,NaN
...,...,...,...,...,...
5153180,76561198326700687,177,493440,http://steamcommunity.com/profiles/76561198326...,NaN
5153185,76561198326700687,177,496920,http://steamcommunity.com/profiles/76561198326...,NaN
5153193,76561198326700687,177,508380,http://steamcommunity.com/profiles/76561198326...,NaN
5153195,76561198326700687,177,510660,http://steamcommunity.com/profiles/76561198326...,NaN


DATA_USERS_SPEND: Se calcula en una columna el gasto total por usuario.

In [12]:
# Creamos un dataframe con las siguientes columnas: user_id, item_count, item_spend.
data_users_spend = data_users.copy()
data_users_spend = data_users[['user_id', 'items_count']].drop_duplicates()

# Calcular el total gastado por usuario y agregarlo como 'item_spend'
data_users_spend['item_spend'] = data_users.groupby('user_id')['price'].sum().reset_index(drop=True)


In [13]:
data_users_spend.head()

,user_id,items_count,item_spend
0,76561197970982479,277,402.77
277,js41637,888,84.95
1165,evcentric,137,2911.18
1302,Riot-Punch,328,74.90
1630,doctr,541,317.81


DATA_USERS_RECOMMEND: Se crea una columna con el % de recomendacion positiva en relacion a todas las recomendaciones hechas por el usuario.

In [14]:
#Se copia el dataframe 'data_users_revs' y se eliminan las columnas que no son necesarias para esta consulta de la API
data_users_recommend = data_users_revs.copy()
data_users_recommend.drop(columns=['item_id','sentiment_analysis', 'rev_date'], inplace=True)
#Se agrega la columna 'true' con el valor 1 si el usuario recomienda el juego y 0 si no lo recomienda.
data_users_recommend['true'] = data_users_recommend['recommend'].apply(lambda x: 1 if x == True else 0)
data_users_recommend['false'] = data_users_recommend['recommend'].apply(lambda x: 1 if x == False else 0)
#Se agrega la columna total true y total false por usuario.
data_users_recommend['total_true'] = data_users_recommend.groupby('user_id')['true'].transform('sum')
data_users_recommend['total_false'] = data_users_recommend.groupby('user_id')['false'].transform('sum')
#Se eliminan las columnas 'recommend', 'true' y 'false'.
data_users_recommend.drop(columns=['recommend', 'true', 'false'], inplace=True)
#Se eliminan los registros duplicados.
data_users_recommend.drop_duplicates(inplace=True)
#Se agrega la columna recommend_%
data_users_recommend['recommend_%'] = (data_users_recommend['total_true'] / (data_users_recommend['total_true'] + data_users_recommend['total_false']))*100
#Se eliminan las columnas 'total_true' y 'total_false'.
data_users_recommend.drop(columns=['total_true', 'total_false'], inplace=True)

In [15]:
data_users_recommend.value_counts()

user_id                         recommend_%
--000--                         100.000000     1
SteamPunkDude227                66.666667      1
Steam-Community-Internet-Whore  100.000000     1
Steam-Community-2002-2006       0.000000       1
StealthChocy                    0.000000       1
                                              ..
76561198081631721               100.000000     1
76561198081622518               100.000000     1
76561198081613852               100.000000     1
76561198081604689               71.428571      1
zzoptimuszz                     100.000000     1
Name: count, Length: 25458, dtype: int64

In [16]:
data_users_recommend.head()

,user_id,recommend_%
0,76561197970982479,100.0
3,js41637,100.0
6,evcentric,100.0
12,doctr,100.0
18,maplemage,100.0


In [17]:
#Se unifican los dataframes 'data_users_spend' y 'data_users_recommend' a traves de la columna 'user_id'.
api_userdata = data_users_spend.merge(data_users_recommend, on='user_id', how='left')

# API_USERDATA

In [18]:
api_userdata.head() #Se utilizaran las columnas user_id	, items_count, item_spend y recommend_% para las consultas de la API.

,user_id,items_count,item_spend,recommend_%
0,76561197970982479,277,402.77,100.0
1,js41637,888,84.95,100.0
2,evcentric,137,2911.18,100.0
3,Riot-Punch,328,74.90,NaN
4,doctr,541,317.81,100.0


# 2do endpoint countreviews

## api_countreviews - Process

DATA_USERS_REVS

In [19]:
data_users_revs.head()

,user_id,item_id,recommend,sentiment_analysis,rev_date
0,76561197970982479,1250,True,1,2011-11-05
1,76561197970982479,22200,True,2,2011-07-15
2,76561197970982479,43110,True,1,2011-04-21
3,js41637,251610,True,1,2014-06-24
4,js41637,227300,True,1,2013-09-08


In [20]:
api_countreviews = data_users_revs.copy() 

# API_COUNTREVIEWS

In [21]:
api_countreviews #Se utilizaran las columnas rev_date y user_id para las consultas de la API.

,user_id,item_id,recommend,sentiment_analysis,rev_date
0,76561197970982479,1250,True,1,2011-11-05
1,76561197970982479,22200,True,2,2011-07-15
2,76561197970982479,43110,True,1,2011-04-21
3,js41637,251610,True,1,2014-06-24
4,js41637,227300,True,1,2013-09-08
...,...,...,...,...,...
59300,76561198312638244,70,True,2,NaN
59301,76561198312638244,362890,True,1,NaN
59302,LydiaMorley,273110,True,1,NaN
59303,LydiaMorley,730,True,2,NaN


# 3er endpoint genre

## api_genre_rank - Process

DATA_USERS_ITEMS

In [22]:
data_users_items.head()

,user_id,items_count,steam_id,user_index_count,item_id,item_name,playtime_forever,playtime_2weeks,user_url
0,76561197970982479,277,76561197970982479,1,10,Counter-Strike,6,0,http://steamcommunity.com/profiles/76561197970...
1,76561197970982479,277,76561197970982479,1,20,Team Fortress Classic,0,0,http://steamcommunity.com/profiles/76561197970...
2,76561197970982479,277,76561197970982479,1,30,Day of Defeat,7,0,http://steamcommunity.com/profiles/76561197970...
3,76561197970982479,277,76561197970982479,1,40,Deathmatch Classic,0,0,http://steamcommunity.com/profiles/76561197970...
4,76561197970982479,277,76561197970982479,1,50,Half-Life: Opposing Force,0,0,http://steamcommunity.com/profiles/76561197970...


DATA_GENRE

In [23]:
data_genre.head()

,item_id,Software Training,Animation &amp; Modeling,Utilities,Simulation,Strategy,Indie,Accounting,Racing,Design &amp; Illustration,...,Sports,Casual,Video Production,Audio Production,Photo Editing,Web Publishing,RPG,Free to Play,Massively Multiplayer,Action
0,761140.0,0,0,0,1,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,643980.0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0
2,670290.0,0,0,0,1,0,1,0,0,0,...,1,1,0,0,0,0,0,1,0,0
3,767400.0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,773570.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
data_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   item_id                    32133 non-null  float64
 1   Software Training          32133 non-null  int64  
 2   Animation &amp; Modeling   32133 non-null  int64  
 3   Utilities                  32133 non-null  int64  
 4   Simulation                 32133 non-null  int64  
 5   Strategy                   32133 non-null  int64  
 6   Indie                      32133 non-null  int64  
 7   Accounting                 32133 non-null  int64  
 8   Racing                     32133 non-null  int64  
 9   Design &amp; Illustration  32133 non-null  int64  
 10  Adventure                  32133 non-null  int64  
 11  Education                  32133 non-null  int64  
 12  Early Access               32133 non-null  int64  
 13  Sports                     32133 non-null  int

In [25]:
#Se unifican los tipos de datos de las 2 tablas a trabajar
data_genre['item_id'] = data_genre['item_id'].astype(int)
data_users_items['item_id'] = data_users_items['item_id'].astype(int) 

In [26]:
api_genre_rank = data_users_items.copy() #Se crea una copia del dataframe para trabajar con el.

In [27]:
#Se eliminan las columnas que no son necesarias para esta consulta de la API
api_genre_rank.drop(columns=['user_id' ,'items_count', 'steam_id', 'user_index_count', 'playtime_2weeks','user_url'], inplace=True)

In [28]:
#Sumar los totales de playtime_forever por item_id
api_genre_rank['playtime_forever'] = api_genre_rank.groupby('item_id')['playtime_forever'].transform('sum')

In [29]:
#Se unifican los dataframes 'api_genre_rank' y 'data_genre' a traves de la columna 'item_id'.
api_genre_rank = api_genre_rank.merge(data_genre, on='item_id', how='left')

In [30]:
#Transformo en float la columna 'playtime_forever'
api_genre_rank['playtime_forever'] = api_genre_rank['playtime_forever'].astype(float)
#Cambiar nombre de columna &amp; por &
api_genre_rank.rename(columns={'Animation &amp; Modeling':'Animation & Modeling'},inplace=True)

In [31]:
api_genre_rank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 25 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   item_id                    int32  
 1   item_name                  object 
 2   playtime_forever           float64
 3   Software Training          float64
 4   Animation & Modeling       float64
 5   Utilities                  float64
 6   Simulation                 float64
 7   Strategy                   float64
 8   Indie                      float64
 9   Accounting                 float64
 10  Racing                     float64
 11  Design &amp; Illustration  float64
 12  Adventure                  float64
 13  Education                  float64
 14  Early Access               float64
 15  Sports                     float64
 16  Casual                     float64
 17  Video Production           float64
 18  Audio Production           float64
 19  Photo Editing              float64
 20  We

In [32]:
#Calcula la columna 'total_playtime' en horas
api_genre_rank['playtime_forever'] = (api_genre_rank['playtime_forever'] / 60).round(2)

In [33]:
api_genre_rank.head()

,item_id,item_name,playtime_forever,Software Training,Animation & Modeling,Utilities,Simulation,Strategy,Indie,Accounting,...,Sports,Casual,Video Production,Audio Production,Photo Editing,Web Publishing,RPG,Free to Play,Massively Multiplayer,Action
0,10,Counter-Strike,289766.92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,20,Team Fortress Classic,16028.37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,30,Day of Defeat,12649.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,40,Deathmatch Classic,2574.77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,50,Half-Life: Opposing Force,12242.70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [34]:
#Se convierten en unicos los registros de las columnas item id y item name.
api_genre_rank = api_genre_rank.drop_duplicates(subset=['item_id', 'item_name'])

In [35]:
#Lista de columnas a multiplicar (excluyendo 'item_id' y 'item_name')
columns_to_multiply = api_genre_rank.columns.difference(['item_id', 'item_name'])

#Multiplicar cada valor de 'playtime_forever' por los valores en las columnas seleccionadas
api_genre_rank[columns_to_multiply] = api_genre_rank[columns_to_multiply].multiply(api_genre_rank['playtime_forever'], axis=0)

In [36]:
#Se eliminan las columnas item_id, item_name y playtime_forever ya que no son necesarias.
api_genre_rank.drop(columns=['item_name','playtime_forever'], inplace=True)

In [37]:
api_genre_rank.tail()

,item_id,Software Training,Animation & Modeling,Utilities,Simulation,Strategy,Indie,Accounting,Racing,Design &amp; Illustration,...,Sports,Casual,Video Production,Audio Production,Photo Editing,Web Publishing,RPG,Free to Play,Massively Multiplayer,Action
4913278,375450,0.0,0.0,0.0,0.00,0.85,0.00,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
5092969,353390,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
5105048,354280,0.0,0.0,0.0,2.73,0.00,2.73,0.0,0.0,0.0,...,0.0,2.73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.73
5119454,433920,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
5145725,485270,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [38]:
#Se elimina la columna 'item_id' ya que no es necesaria, para esta consulta de la API.
api_genre_rank.drop(columns=['item_id'], inplace=True)

In [39]:
def sumar_columnas(df):
    #Inicializa una lista para almacenar los resultados
    resultados = []

    #Itera a través de las columnas del DataFrame de entrada
    for columna in df.columns:
        #Calcula la suma de los valores en la columna actual
        total = df[columna].sum()

        #Agrega los resultados a nuestra lista
        resultados.append({"genre": columna, "hr_total_playtime": total})
    # Convierte la lista de resultados en un DataFrame al final
    result_df = pd.DataFrame(resultados)
    return result_df

In [40]:
api_genre_rank = sumar_columnas(api_genre_rank)

Reordenamos el dataframe

In [41]:
#Ordenar de mayor a menor
api_genre_rank = api_genre_rank.sort_values(by='hr_total_playtime', ascending=False)
#Se agrega la columna 'rank' con el ranking de los generos.
api_genre_rank['rank'] = api_genre_rank['hr_total_playtime'].rank(ascending=False)
#Se convierte la columna 'rank' a tipo int.
api_genre_rank['rank'] = api_genre_rank['rank'].astype(int)
#Se reorganizan las columnas del dataframe.
api_genre_rank = api_genre_rank[['rank', 'genre', 'hr_total_playtime']]

In [42]:
api_genre_rank.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, 21 to 6
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rank               22 non-null     int32  
 1   genre              22 non-null     object 
 2   hr_total_playtime  22 non-null     float64
dtypes: float64(1), int32(1), object(1)
memory usage: 616.0+ bytes


# API_GENRE_RANK

In [43]:
api_genre_rank #Se utilizaran las columnas rank, genre y hr_total_playtime para las consultas de la API.

,rank,genre,hr_total_playtime
21,1,Action,51892709.82
5,2,Indie,24910373.20
18,3,RPG,17350378.64
9,4,Adventure,15166585.14
3,5,Simulation,14460771.61
4,6,Strategy,10989397.24
19,7,Free to Play,10179215.71
20,8,Massively Multiplayer,7443234.64
13,9,Casual,4203880.90
11,10,Early Access,2645021.16


# 4to endpoint userforgenre

## api_userforgenre - Process

DATA_USERS_ITEMS

In [88]:
data_users_items.head()

,user_id,items_count,steam_id,user_index_count,item_id,item_name,playtime_forever,playtime_2weeks,user_url
0,76561197970982479,277,76561197970982479,1,10,Counter-Strike,6,0,http://steamcommunity.com/profiles/76561197970...
1,76561197970982479,277,76561197970982479,1,20,Team Fortress Classic,0,0,http://steamcommunity.com/profiles/76561197970...
2,76561197970982479,277,76561197970982479,1,30,Day of Defeat,7,0,http://steamcommunity.com/profiles/76561197970...
3,76561197970982479,277,76561197970982479,1,40,Deathmatch Classic,0,0,http://steamcommunity.com/profiles/76561197970...
4,76561197970982479,277,76561197970982479,1,50,Half-Life: Opposing Force,0,0,http://steamcommunity.com/profiles/76561197970...


DATA_GENRE

In [89]:
data_genre.head()

,item_id,Software Training,Animation &amp; Modeling,Utilities,Simulation,Strategy,Indie,Accounting,Racing,Design &amp; Illustration,...,Sports,Casual,Video Production,Audio Production,Photo Editing,Web Publishing,RPG,Free to Play,Massively Multiplayer,Action
0,761140,0,0,0,1,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,643980,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0
2,670290,0,0,0,1,0,1,0,0,0,...,1,1,0,0,0,0,0,1,0,0
3,767400,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,773570,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
data_aux_user = data_users_items.copy() #Se crea una copia del dataframe para trabajar con el.
#Se eliminan las columnas que no son necesarias para esta consulta de la API
data_aux_user.drop(columns=['item_name', 'user_index_count', 'items_count', 'playtime_2weeks'], inplace=True) 

In [91]:
#Se convierte la columna 'item_id' a tipo int, para unificarla posteriormente.
data_aux_user['item_id'] = data_aux_user['item_id'].astype(int)

In [92]:
#Se reorganizan las columnas del dataframe.
data_aux_user = data_aux_user[['user_id','steam_id', 'user_url','item_id', 'playtime_forever']]

In [93]:
data_aux_user.head()

,user_id,steam_id,user_url,item_id,playtime_forever
0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,6
1,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,0
2,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,7
3,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,0
4,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,0


In [94]:
data_playtime = data_aux_user.copy() #Se crea una copia del dataframe para trabajar con el.
#Se eliminan las columnas que no son necesarias para esta consulta de la API
data_playtime.drop(columns=['user_url', 'user_id', 'steam_id'], inplace=True)

In [95]:
data_aux_genre = data_genre.copy() #Se crea una copia del dataframe para trabajar con el.

In [96]:
#Se convierte la columna 'item_id' a tipo int, para unificarla posteriormente.
data_aux_genre['item_id'] = data_aux_genre['item_id'].astype(int)

In [97]:
data_aux_genre.head()

,item_id,Software Training,Animation &amp; Modeling,Utilities,Simulation,Strategy,Indie,Accounting,Racing,Design &amp; Illustration,...,Sports,Casual,Video Production,Audio Production,Photo Editing,Web Publishing,RPG,Free to Play,Massively Multiplayer,Action
0,761140,0,0,0,1,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,643980,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0
2,670290,0,0,0,1,0,1,0,0,0,...,1,1,0,0,0,0,0,1,0,0
3,767400,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,773570,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
#Se unifican los dataframes 'data_playtime' y 'data_aux_genre' a traves de la columna 'item_id'.
data_playtime = data_playtime.merge(data_aux_genre, on='item_id', how='left')

In [99]:
data_playtime.head()

,item_id,playtime_forever,Software Training,Animation &amp; Modeling,Utilities,Simulation,Strategy,Indie,Accounting,Racing,...,Sports,Casual,Video Production,Audio Production,Photo Editing,Web Publishing,RPG,Free to Play,Massively Multiplayer,Action
0,10,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,20,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,30,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,40,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,50,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [100]:
#Sumar los totales de playtime_forever por item_id
data_playtime['playtime_forever'] = data_playtime.groupby('item_id')['playtime_forever'].transform('sum')

In [101]:
#Transformo en float la columna 'playtime_forever'
data_playtime['playtime_forever'] = data_playtime['playtime_forever'].astype(float)

In [102]:
#Calcula la columna 'total_playtime' en horas
data_playtime['playtime_forever'] = (data_playtime['playtime_forever'] / 60).round(2)

In [103]:
#Se convierte en unicos los registros de la columna item id.
data_playtime = data_playtime.drop_duplicates(subset=['item_id'])

In [104]:
#Lista de columnas a multiplicar (excluyendo 'item_id')
columns_to_multiply = data_playtime.columns.difference(['item_id'])

#Multiplicar cada valor de 'playtime_forever' por los valores en las columnas seleccionadas
data_playtime[columns_to_multiply] = data_playtime[columns_to_multiply].multiply(data_playtime['playtime_forever'], axis=0)

In [105]:
#Se elimina la columna playtime_forever ya que no es mas necesaria.
data_playtime.drop(columns=['playtime_forever'], inplace=True)

In [106]:
data_playtime.tail()

,item_id,Software Training,Animation &amp; Modeling,Utilities,Simulation,Strategy,Indie,Accounting,Racing,Design &amp; Illustration,...,Sports,Casual,Video Production,Audio Production,Photo Editing,Web Publishing,RPG,Free to Play,Massively Multiplayer,Action
4913278,375450,0.0,0.0,0.0,0.00,0.85,0.00,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
5092969,353390,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
5105048,354280,0.0,0.0,0.0,2.73,0.00,2.73,0.0,0.0,0.0,...,0.0,2.73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.73
5119454,433920,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
5145725,485270,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [107]:
data_aux_user.head()

,user_id,steam_id,user_url,item_id,playtime_forever
0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,6
1,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,0
2,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,7
3,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,0
4,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,0


In [108]:
#Se crea una copia del dataframe para trabajar con el.
api_userforgenre = data_aux_user.copy() 
#Se eliminan las columnas que no son necesarias para esta consulta de la API
api_userforgenre = api_userforgenre.drop(columns=['playtime_forever'])

In [109]:
api_userforgenre.head()

,user_id,steam_id,user_url,item_id
0,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10
1,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20
2,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30
3,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40
4,76561197970982479,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50


In [110]:
#Se unifican los dataframes 'data_aux_user' y 'data_aux_genre' a traves de la columna 'item_id'.
api_userforgenre = api_userforgenre.merge(data_playtime, on='item_id', how='left')

In [111]:
#Se elimina la columna 'item_id' ya que no es necesaria, para esta consulta de la API.
api_userforgenre.drop(columns=['item_id'], inplace=True)

In [119]:
#Se vuelven unicos los valores de las columnas user_id y user_url, tambien se suman los valores totales de cada columna decada genero
api_userforgenre = api_userforgenre.groupby(['user_id', 'user_url','steam_id']).sum().reset_index()

# API_USERFORGENRE

In [120]:
api_userforgenre.head()

,user_id,user_url,steam_id,Software Training,Animation &amp; Modeling,Utilities,Simulation,Strategy,Indie,Accounting,...,Sports,Casual,Video Production,Audio Production,Photo Editing,Web Publishing,RPG,Free to Play,Massively Multiplayer,Action
0,--000--,http://steamcommunity.com/id/--000--,76561198084750369,0.0,0.0,0.0,8439736.52,613108.19,13060129.43,0.0,...,59853.28,2172879.38,0.0,0.0,0.0,0.0,4452748.17,5951598.04,1637448.23,23817963.35
1,--ace--,http://steamcommunity.com/id/--ace--,76561198113770873,0.0,0.0,0.0,8049501.57,622683.15,9887946.49,0.0,...,914.87,2205024.54,0.0,0.0,0.0,0.0,1235976.60,2216861.23,292643.70,15849500.51
2,--ionex--,http://steamcommunity.com/id/--ionex--,76561198101468258,0.0,0.0,0.0,949353.79,597648.65,4897242.31,0.0,...,0.00,1704347.91,0.0,0.0,0.0,0.0,4156881.48,2634277.14,1019584.38,19989320.61
3,-2SV-vuLB-Kg,http://steamcommunity.com/id/-2SV-vuLB-Kg,76561197999208914,0.0,0.0,0.0,1867.23,300135.72,56333.49,0.0,...,444.32,18969.51,0.0,0.0,0.0,0.0,126551.45,43761.01,31196.52,13282519.28
4,-404PageNotFound-,http://steamcommunity.com/id/-404PageNotFound-,76561198074247623,0.0,0.0,0.0,9780123.98,2592439.38,13440938.10,0.0,...,96065.82,1627084.27,0.0,0.0,0.0,0.0,5457879.65,7202424.49,5413522.97,32375201.02


# Revision final

In [115]:
api_userdata.info() #ok

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70912 entries, 0 to 70911
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      70912 non-null  object 
 1   items_count  70912 non-null  int64  
 2   item_spend   708 non-null    float64
 3   recommend_%  22613 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.2+ MB


In [116]:
api_countreviews.info() #ok

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   item_id             59305 non-null  int64 
 2   recommend           59305 non-null  bool  
 3   sentiment_analysis  59305 non-null  int64 
 4   rev_date            49186 non-null  object
dtypes: bool(1), int64(2), object(2)
memory usage: 1.9+ MB


In [117]:
api_genre_rank.info() #ok

<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, 21 to 6
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rank               22 non-null     int32  
 1   genre              22 non-null     object 
 2   hr_total_playtime  22 non-null     float64
dtypes: float64(1), int32(1), object(1)
memory usage: 616.0+ bytes


In [121]:
api_userforgenre.info() #ok

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70912 entries, 0 to 70911
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   user_id                    70912 non-null  object 
 1   user_url                   70912 non-null  object 
 2   steam_id                   70912 non-null  int64  
 3   Software Training          70912 non-null  float64
 4   Animation &amp; Modeling   70912 non-null  float64
 5   Utilities                  70912 non-null  float64
 6   Simulation                 70912 non-null  float64
 7   Strategy                   70912 non-null  float64
 8   Indie                      70912 non-null  float64
 9   Accounting                 70912 non-null  float64
 10  Racing                     70912 non-null  float64
 11  Design &amp; Illustration  70912 non-null  float64
 12  Adventure                  70912 non-null  float64
 13  Education                  70912 non-null  flo

## Guardado

In [123]:
#Se guardan en formato csv.
api_userdata.to_csv('Rest API/Datasets CSV - API/api_userdata.csv', index=False)          #Peso 2.2+MB 
api_countreviews.to_csv('Rest API/Datasets CSV - API/api_countreviews.csv', index=False)  #Peso 1.9+MB
api_genre_rank.to_csv('Rest API/Datasets CSV - API/api_genre_rank.csv', index=False)      #Peso 616bytes
api_userforgenre.to_csv('Rest API/Datasets CSV - API/api_userforgenre.csv', index=False)  #Peso 13.5+MB
#api           .to_csv('Rest API/Datasets CSV - API/           .csv', index=False)
#api           .to_csv('Rest API/Datasets CSV - API/           .csv', index=False)